# A1111 Model Installer — SD 1.5

Installs **Stable Diffusion 1.5** models and related assets into the WebUI data directory:

- Checkpoints → `/workspace/a1111/models/Stable-diffusion/`
- VAE → `/workspace/a1111/models/VAE/`
- ControlNet → `/workspace/a1111/models/ControlNet/`
- ADetailer → `/workspace/a1111/models/adetailer/`
- ESRGAN Upscalers → `/workspace/a1111/models/ESRGAN/`

**How it works**
- Choose which items to install using checkboxes.
- Paste additional checkpoint URLs if desired.
- Re-running skips already-downloaded files.

> If a model requires login on Hugging Face, set `HF_TOKEN` in your RunPod template.

In [ ]:
# --- Paths / setup ---
from pathlib import Path
import os

ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = ROOT / "models/Stable-diffusion"
VAE_DIR  = ROOT / "models/VAE"
CN_DIR   = ROOT / "models/ControlNet"
AD_DIR   = ROOT / "models/adetailer"
ESRGAN_DIR = ROOT / "models/ESRGAN"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, AD_DIR, ESRGAN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Model directories ready:")
print(f"✓ Checkpoints : {CKPT_DIR}")
print(f"✓ VAE         : {VAE_DIR}")
print(f"✓ ControlNet  : {CN_DIR}")
print(f"✓ ADetailer   : {AD_DIR}")
print(f"✓ ESRGAN      : {ESRGAN_DIR}")

## Helpers (progress + smart filenames)
Supports **Hugging Face** (repo + filename) and **direct URLs** (e.g. Civitai) with correct filenames from headers.

In [ ]:
# --- Helper functions (progress + smart filenames) ---
import os, re, shutil, requests
from tqdm import tqdm
from pathlib import Path
from huggingface_hub import hf_hub_download

def human(n: int) -> str:
    for u in ["B","KB","MB","GB"]:
        if n < 1024: return f"{n:.1f} {u}"
        n /= 1024
    return f"{n:.1f} TB"

def _save_stream_to(path: Path, response: requests.Response):
    path.parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        print("✔ Exists:", path); return path
    total = int(response.headers.get("Content-Length", 0))
    tmp = path.with_suffix(path.suffix + ".part")
    with open(tmp, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=path.name, ncols=80) as bar:
        for chunk in response.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk); bar.update(len(chunk))
    tmp.replace(path)
    print(f"✔ Saved: {path} ({human(path.stat().st_size)})")
    return path

def download_direct(url: str, dest_dir: Path) -> Path:
    """
    Download from a direct URL (e.g. Civitai). Uses Content-Disposition filename when available.
    Saves into dest_dir / <resolved-filename>.
    """
    dest_dir.mkdir(parents=True, exist_ok=True)
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()

    # Prefer filename from headers (Civitai sets this)
    cd = r.headers.get("Content-Disposition", "")
    m = re.search(r'filename="?([^";]+)"?', cd)
    if m:
        filename = m.group(1)
    else:
        filename = url.split("/")[-1].split("?")[0]
        if not (filename.endswith(".safetensors") or filename.endswith(".ckpt") or "." in filename):
            filename += ".safetensors"

    dest = dest_dir / filename
    return _save_stream_to(dest, r)

def download_to(url: str, dest: Path) -> Path:
    """Download to an explicit full path (use for files with known names)."""
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()
    return _save_stream_to(dest, r)

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str | None = None) -> Path:
    """Download from Hugging Face with resume and optional renaming."""
    dest = dest_dir / (dest_name or filename)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=os.environ.get("HF_TOKEN"),
        resume_download=True
    )
    shutil.copy2(path, dest)
    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")
    return dest

## Pick what to install
- Paste SD 1.5 **checkpoint URLs** below (Civitai, HF, etc.). One per line.
- **VAE** is optional (unchecked by default).
- **ControlNet SD15**: pre-checked defaults are Canny, SoftEdge, OpenPose.

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# Optional components
vae_box = W.Checkbox(description="Install SD1.5 MSE VAE", value=False)
ad_box  = W.Checkbox(description="Install ADetailer face model", value=True)
esr_box = W.Checkbox(description="Install 4x-UltraSharp ESRGAN", value=True)

# ControlNet SD15 options
ncn_labels = [
    ("Canny", "lllyasviel/control_v11p_sd15_canny"),
    ("SoftEdge", "lllyasviel/control_v11p_sd15_softedge"),
    ("OpenPose", "lllyasviel/control_v11p_sd15_openpose"),
    ("Depth", "lllyasviel/control_v11p_sd15_depth"),
    ("NormalBae", "lllyasviel/control_v11p_sd15_normalbae")
]
prechecked = {"Canny", "SoftEdge", "OpenPose"}
cn_boxes = [W.Checkbox(description=l, value=(l in prechecked)) for l, _ in cn_labels]

ckpt_area = W.Textarea(
    placeholder="Paste SD1.5 checkpoint URLs (.safetensors, .ckpt), one per line.",
    layout=W.Layout(width="100%", height="120px")
)

button = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def run_download(_):
    out.clear_output()
    with out:
        errs = []
        # VAE
        if vae_box.value:
            try:
                download_hf("stabilityai/sd-vae-ft-mse", "vae-ft-mse-840000-ema-pruned.safetensors", VAE_DIR)
            except Exception as e:
                errs.append(("VAE", e))

        # ADetailer
        if ad_box.value:
            try:
                download_to("https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8n.pt", AD_DIR / "face_yolov8n.pt")
            except Exception as e:
                errs.append(("ADetailer", e))

        # ESRGAN
        if esr_box.value:
            try:
                download_hf("Coyote-A/4x-UltraSharp", "4x-UltraSharp.pth", ESRGAN_DIR)
            except Exception as e:
                errs.append(("ESRGAN", e))

        # ControlNet SD15
        for box, (label, repo) in zip(cn_boxes, cn_labels):
            if box.value:
                try:
                    download_hf(repo, "diffusion_pytorch_model.safetensors", CN_DIR, f"control_v11p_sd15_{label.lower()}.safetensors")
                except Exception as e:
                    errs.append((label, e))

        # User checkpoints (smart filename from headers)
        for url in [u.strip() for u in ckpt_area.value.splitlines() if u.strip()]:
            try:
                download_direct(url, CKPT_DIR)
            except Exception as e:
                errs.append((url, e))

        print("\n=== Summary ===")
        if errs:
            for name, e in errs:
                print(f"✖ {name}: {e}")
        else:
            print("✔ All downloads completed successfully.")

button.on_click(run_download)

display(Markdown("### Select Models to Download"))
display(ckpt_area)
display(vae_box, ad_box, esr_box)
display(Markdown("#### ControlNet (SD1.5):"))
for c in cn_boxes:
    display(c)
display(button, out)

## Restart WebUI (required after installing)
Restarts A1111 under **supervisord** so new models are recognized.

In [ ]:
import subprocess, time, shlex
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print(out.stdout.strip())
    else:
        print("Supervisorctl unavailable, killing process…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI terminated, Supervisor will auto-restart it")
    print("Wait 10–15 seconds, then refresh WebUI (7860).")
except Exception as e:
    print("⚠️ Restart failed:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")